In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [3]:
import torch
import kagglehub
import pandas as pd
from data_loader import CustomDataLoader
from train import TrainingLoop
from eval import Evaluate
import torch.nn.functional as F

## Downoad Dataset

In [26]:
# Download latest version of test dataset
test_path = kagglehub.dataset_download("mdismielhossenabir/sentiment-analysis")

print("Path to dataset files:", test_path)

Path to dataset files: /Users/jaylodha/.cache/kagglehub/datasets/mdismielhossenabir/sentiment-analysis/versions/1


In [21]:
# Download the dataset- this will have the train and val sets
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")

print("Path to dataset files:", path)

Path to dataset files: /Users/jaylodha/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2


## Read data into CSV files

#### load the train and val dfs first

In [22]:
train_df = pd.read_csv(f"{path}/twitter_training.csv", header = None)
val_df = pd.read_csv(f"{path}/twitter_validation.csv", header = None)

In [23]:
train_df.head()

0            1         2  \
0  2401  Borderlands  Positive   
1  2401  Borderlands  Positive   
2  2401  Borderlands  Positive   
3  2401  Borderlands  Positive   
4  2401  Borderlands  Positive   

                                                   3  
0  im getting on borderlands and i will murder yo...  
1  I am coming to the borders and I will kill you...  
2  im getting on borderlands and i will kill you ...  
3  im coming on borderlands and i will murder you...  
4  im getting on borderlands 2 and i will murder ...

#### load the test df- also apply some post processing to esnure consistent column names

In [27]:
test_df = pd.read_csv(f"{test_path}/sentiment_analysis.csv")

In [28]:
# Apply camel casing
test_df['sentiment'] = test_df['sentiment'].str.capitalize()

In [29]:
test_df.rename(columns={'sentiment': 2}, inplace=True)
test_df.rename(columns={'text': 3}, inplace=True)

In [31]:
test_df.head()

Year  Month  Day Time of Tweet  \
0  2018      8   18       morning   
1  2018      8   18          noon   
2  2017      8   18         night   
3  2022      6    8       morning   
4  2022      6    8          noon   

                                                   3         2     Platform  
0              What a great day!!! Looks like dream.  Positive    Twitter    
1     I feel sorry, I miss you here in the sea beach  Positive    Facebook   
2                                     Don't angry me  Negative     Facebook  
3  We attend in the class just for listening teac...  Negative    Facebook   
4                  Those who want to go, let them go  Negative   Instagram

## Load data in CustomDataLoader

In [32]:
text_col = 3    # Column index for input text
label_col = 2   # Column index for labels
batch_size = 8 # Batch size for training and validation

# Initialize CustomDataLoader
custom_loader = CustomDataLoader(train_df, val_df, test_df, text_col, label_col, batch_size)

# Get train, validation and test loaders
train_loader = custom_loader.get_train_loader(shuffle=True)
val_loader = custom_loader.get_val_loader(shuffle=True)
test_loader = custom_loader.get_test_loader(shuffle=False)

## Define Model Params

In [22]:
model_params = {
    "vocab_size": train_loader.dataset.vocab_size,
    "num_embeddings": 64,
    "block_size": train_loader.dataset.block_size,
    "num_heads": 4,
    "num_layers": 4,
    "output_classes": len(train_loader.dataset.labels_lookup_dict),
    "dropout": 0.2,
    "device": 'cuda' if torch.cuda.is_available() else 'cpu'
}

## Define Training Params

In [23]:
train_params = {
    "num_epochs": 100,
    "eval_interval": 10,
    "eval_iters": 10,
    "learning_rate": 0.01
}

## Training Loop

In [24]:
save_models_path = "./models_v1"
TrainingLoop(model_params, train_params).train(train_loader, val_loader, save_models_path, resume_path="/home/adityadev/GPTDecoder/models_v1/best_model.pth")

/home/adityadev/GPTDecoder/train.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(load_path, map_location=self.device)  # Ensure checkpoint is lo

## Evaluate Best model- load it from checkpoint

#### Evaluate first on the validation set

In [25]:
val_loader = custom_loader.get_val_loader(shuffle=False)
best_model_path = "/home/adityadev/GPTDecoder/models_v1/best_model.pth"
label_mapping = train_loader.dataset.reverse_labels_lookup_dict

report = Evaluate(model_params, best_model_path).evaluate(val_loader, label_mapping)

/home/adityadev/GPTDecoder/eval.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.best_model_path, map_location=self.device)
2024-11-23 20:35

In [26]:
print(report)

              precision    recall  f1-score   support

  Irrelevant     0.0000    0.0000    0.0000       172
    Negative     0.0000    0.0000    0.0000       266
     Neutral     0.3067    0.2561    0.2792       285
    Positive     0.2754    0.7545    0.4035       277

    accuracy                         0.2820      1000
   macro avg     0.1455    0.2527    0.1707      1000
weighted avg     0.1637    0.2820    0.1913      1000



#### Also evaluate on the test set

In [ ]:
report = Evaluate(model_params, best_model_path).evaluate(test_loader, label_mapping)

## Infer on Raw text

In [34]:
model = Evaluate(model_params, best_model_path).model

/home/adityadev/GPTDecoder/eval.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.best_model_path, map_location=self.device)
2024-11-23 20:42

In [ ]:
def infer_on_raw_text(raw_text: str) -> str:

  encoded_inp = torch.tensor(train_loader.dataset.encode_text(raw_text), dtype=torch.long, device=model_params.get('device')).unsqueeze(0)
  logits = model(encoded_inp)
  probs = F.softmax(logits[0], dim=-1)
  # Find the index of the largest element
  max_index = torch.argmax(probs, dim=1)

  label = label_mapping[max_index.item()]
  return label

In [36]:
text = "I'm very happy today"
infer_on_raw_text(text)

'Positive'